# Disentangled RNN Analysis
In this notebook, you can see how the latent variables are maintained inside the disentangled RNN.

In [1]:
from src import disrnn
from src import switch_utils
import os
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
model_dir = "/Users/michaelcondon/workspaces/pbm_group2/disentangled_rnns/models/"

# choose mouse, beta and run time
train_prop = 0.7
param = 0.003
cv = f'{train_prop*100:.0f}-{(1-train_prop)*100:.0f}'
train_dt = "2025-04-10_12-51"
batch_size = 30

model_shape = {'dis_latent_size': 5,
                'update_mlp_shape': (5,5,5),
                'choice_mlp_shape': (2,2),
                'obs_size': 2,
                'target_size': 2}

params_file = os.path.join(model_dir, f"params_{param:.0e}_{cv}_{train_dt}.json")
params = switch_utils.model_loader(params_file=params_file)

In [ ]:
# Eval mode runs the network with no noise
def make_network_eval():
  return disrnn.HkDisRNN(update_mlp_shape=model_shape['update_mlp_shape'],
                        choice_mlp_shape=model_shape['choice_mlp_shape'],
                        latent_size=model_shape['dis_latent_size'],
                        obs_size=2, 
                        target_size=2,
                        eval_mode=True)


disrnn.plot_bottlenecks(params, sort_latents=True)
plt.show()
disrnn.plot_update_rules(params, make_network_eval)
plt.show()

In [3]:
data_dir = '/Users/michaelcondon/workspaces/pbm_group2/disentangled_rnns/data/processed'
model_dir = "/Users/michaelcondon/workspaces/pbm_group2/disentangled_rnns/models/"

# choose mouse, beta and run time
model_name = 'disrnn'
train_prop = 0.7
param = 0.003
cv = f'{train_prop*100:.0f}-{(1-train_prop)*100:.0f}'
train_dt = "2025-04-18_00-24"
split_dt = "2025-04-17_10-28"
batch_size = 64

model_shape = {'dis_latent_size': 5,
                'tiny_latent_size': 2,
                'update_mlp_shape': (5,5,5),
                'choice_mlp_shape': (2,2),
                'obs_size': 2,
                'target_size': 2}

param_dict = {'disrnn': f'{param:.0e}', 'rnn': f'{param}'}
params_file = os.path.join(model_dir, f"params_{model_name}_{param_dict[model_name]}_{cv}_{train_dt}.json")
loss_file = os.path.join(model_dir, f"loss_{model_name}_{param_dict[model_name]}_{cv}_{train_dt}.csv")

df_tr = pd.read_csv(os.path.join(data_dir, f"train_df_{cv}_{split_dt}.csv"))
df_va = pd.read_csv(os.path.join(data_dir, f"validation_df_{cv}_{split_dt}.csv"))

params, losses = switch_utils.model_loader(params_file=params_file, loss_file=loss_file)

ds_tr = switch_utils.get_dataset(df_tr, batch_size)
ds_va = switch_utils.get_dataset(df_va, batch_size)


def make_network_eval():
    return disrnn.HkDisRNN(update_mlp_shape=model_shape['update_mlp_shape'],
                            choice_mlp_shape=model_shape['choice_mlp_shape'],
                            latent_size=model_shape['dis_latent_size'],
                            obs_size=model_shape['obs_size'], 
                            target_size=model_shape['target_size'],
                            eval_mode=True)




FileNotFoundError: [Errno 2] No such file or directory: '/Users/michaelcondon/workspaces/pbm_group2/disentangled_rnns/models/params_3e-03_70-30_2025-04-18_00-24.json'

In [ ]:
disrnn.plot_bottlenecks(params, sort_latents=True)
plt.show()
disrnn.plot_update_rules(params, make_network_eval)
plt.show()